# Learning Objectives
In this notebook, you will learn Spark Dataframe APIs.

# Question List

Solve the following questions using Spark Dataframe APIs

### Join

1. easy - https://pgexercises.com/questions/joins/simplejoin.html
2. easy - https://pgexercises.com/questions/joins/simplejoin2.html
3. easy - https://pgexercises.com/questions/joins/self2.html 
4. medium - https://pgexercises.com/questions/joins/threejoin.html (three join)
5. medium - https://pgexercises.com/questions/joins/sub.html (subquery and join)

### Aggregation

1. easy - https://pgexercises.com/questions/aggregates/count3.html Group by order by
2. easy - https://pgexercises.com/questions/aggregates/fachours.html group by order by
3. easy - https://pgexercises.com/questions/aggregates/fachoursbymonth.html group by with condition 
4. easy - https://pgexercises.com/questions/aggregates/fachoursbymonth2.html group by multi col
5. easy - https://pgexercises.com/questions/aggregates/members1.html count distinct
6. med - https://pgexercises.com/questions/aggregates/nbooking.html group by multiple cols, join

### String & Date

1. easy - https://pgexercises.com/questions/string/concat.html format string
2. easy - https://pgexercises.com/questions/string/case.html WHERE + string function
3. easy - https://pgexercises.com/questions/string/reg.html WHERE + string function
4. easy - https://pgexercises.com/questions/string/substr.html group by, substr
5. easy - https://pgexercises.com/questions/date/series.html generate ts
6. easy - https://pgexercises.com/questions/date/bookingspermonth.html extract month from ts

### JOIN
### Question 1

How can you produce a list of the start times for bookings by members named 'David Farrell'?

https://pgexercises.com/questions/joins/simplejoin.html

In [0]:
# Write you solution here
# hint: you might need to re-run `0 - ETL pgexercieses CSV files` notebook to init tables

bookings_df = spark.sql("select * from bookings")
members_df = spark.sql("select * from members")
df = bookings_df.join(members_df, bookings_df.memid == members_df.memid,"inner")
df.filter((df.firstname == 'David') & (df.surname == 'Farrell')).select('starttime').show()

+-------------------+
|          starttime|
+-------------------+
|2012-09-18 09:00:00|
|2012-09-18 17:30:00|
|2012-09-18 13:30:00|
|2012-09-18 20:00:00|
|2012-09-19 09:30:00|
|2012-09-19 15:00:00|
|2012-09-19 12:00:00|
|2012-09-20 15:30:00|
|2012-09-20 11:30:00|
|2012-09-20 14:00:00|
|2012-09-21 10:30:00|
|2012-09-21 14:00:00|
|2012-09-22 08:30:00|
|2012-09-22 17:00:00|
|2012-09-23 08:30:00|
|2012-09-23 17:30:00|
|2012-09-23 19:00:00|
|2012-09-24 08:00:00|
|2012-09-24 16:30:00|
|2012-09-24 12:30:00|
+-------------------+
only showing top 20 rows



### Question 2
How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.

https://pgexercises.com/questions/joins/simplejoin2.html

In [0]:

# Define the SQL query
query = """
SELECT bookings.starttime AS start, facilities.name AS name
FROM bookings
INNER JOIN facilities 
ON facilities.facid = bookings.facid
WHERE facilities.name IN ('Tennis Court 2', 'Tennis Court 1') 
AND TO_DATE(bookings.starttime) = '2012-09-21'
ORDER BY bookings.starttime
"""

# Execute the query
df = spark.sql(query)
display(df)

+-------------------+--------------+
|              start|          name|
+-------------------+--------------+
|2012-09-21 08:00:00|Tennis Court 1|
|2012-09-21 08:00:00|Tennis Court 2|
|2012-09-21 09:30:00|Tennis Court 1|
|2012-09-21 10:00:00|Tennis Court 2|
|2012-09-21 11:30:00|Tennis Court 2|
|2012-09-21 12:00:00|Tennis Court 1|
|2012-09-21 13:30:00|Tennis Court 1|
|2012-09-21 14:00:00|Tennis Court 2|
|2012-09-21 15:30:00|Tennis Court 1|
|2012-09-21 16:00:00|Tennis Court 2|
|2012-09-21 17:00:00|Tennis Court 1|
|2012-09-21 18:00:00|Tennis Court 2|
+-------------------+--------------+



### Question 3
How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).

https://pgexercises.com/questions/joins/threejoin.html

In [0]:
# Define sql query
query = """
select members.firstname as memfname, members.surname as memsname,
recommender.firstname as recfname, recommender.surname as recsname
from members
left join members as recommender
on members.recommendedby=recommender.memid
order by memsname,memfname
"""
# Execute the query
df = spark.sql(query)
display(df)


memfname,memsname,recfname,recsname
Florence,Bader,Ponder,Stibbons
Anne,Baker,Ponder,Stibbons
Timothy,Baker,Jemima,Farrell
Tim,Boothe,Tim,Rownam
Gerald,Butters,Darren,Smith
Joan,Coplin,Timothy,Baker
Erica,Crumpet,Tracy,Smith
Nancy,Dare,Janice,Joplette
David,Farrell,null,null
Jemima,Farrell,null,null


### Question 4
How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

https://pgexercises.com/questions/joins/sub.html

In [0]:
# Define the SQL query
query = """
select distinct firstname|| ' ' ||surname as member, facilities.name as facility
from members
inner join bookings
on members.memid=bookings.memid
inner join facilities 
on bookings.facid = facilities.facid
where facilities.name in ('Tennis Court 2','Tennis Court 1')
order by member, facility
"""

# Execute the query
df = spark.sql(query)
display(df)

member,facility
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Burton Tracy,Tennis Court 1
Burton Tracy,Tennis Court 2
Charles Owen,Tennis Court 1
Charles Owen,Tennis Court 2
Darren Smith,Tennis Court 2
David Farrell,Tennis Court 1
David Farrell,Tennis Court 2
David Jones,Tennis Court 1


### Question 5
How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.

https://pgexercises.com/questions/joins/sub.html

In [0]:
# Define the SQL query
query = """
select distinct members.firstname || ' ' || members.surname as member,
	(select distinct recommender.firstname || ' ' || recommender.surname as recommender
 		from members as recommender
 		where recommender.memid=members.recommendedby
 	)
 	from members
 order by member
"""

# Execute the query
df = spark.sql(query)
display(df)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1617139453244351>:13
      2 query = """
      3 select distinct members.firstname || ' ' || members.surname as member,
      4 	(select distinct recommender.firstname || ' ' || recommender.surname as recommender
   (...)
      9  order by member
     10 """
     12 # Execute the query
---> 13 df = spark.sql(query)
     14 display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:

### Aggregation
### Question 1
Produce a count of the number of recommendations each member has made. Order by member ID.
https://pgexercises.com/questions/aggregates/count3.html

In [0]:
# Define the SQL query
query = """
select recommender.memid as recommendedby, count(recommender.memid) as count
from members
left join members as recommender
on members.recommendedby=recommender.memid
where recommender.memid is not null
group by recommender.memid
order by recommender.memid
"""

# Execute the query
df = spark.sql(query)
display(df)

recommendedby,count
1,5
11,1
13,2
15,1
16,1
2,3
20,1
3,1
30,1
4,2


### Question 2
Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.

https://pgexercises.com/questions/aggregates/fachours.html

In [0]:
# Define the SQL query
query = """
select bookings.facid, sum(slots) as `total slots`
from bookings
inner join facilities
on bookings.facid=facilities.facid
group by bookings.facid
order by bookings.facid
"""

# Execute the query
df = spark.sql(query)
display(df)

facid,total slots
0,1320
1,1278
2,1209
3,830
4,1404
5,228
6,1104
7,908
8,911


### Question 3

Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.

https://pgexercises.com/questions/aggregates/fachoursbymonth.html

In [0]:
# Define the SQL query
query = """
select bookings.facid, sum(slots) as `Total Slots`
from bookings

where bookings.starttime between '2012-09-01' and '2012-10-1'
group by bookings.facid
order by sum(slots)
"""

# Execute the query
df = spark.sql(query)
display(df)

facid,Total Slots
5,122
3,422
7,426
8,471
6,540
2,570
1,588
0,591
4,648


### Question 4

Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.

https://pgexercises.com/questions/aggregates/fachoursbymonth2.html

In [0]:
# Define the SQL query
query = """
select bookings.facid, extract(month from starttime) as month, sum(slots) as `Total Slots`
from bookings
where extract(year from starttime)=2012
group by bookings.facid, extract(month from starttime)
order by bookings.facid, extract(month from starttime)
"""

# Execute the query
df = spark.sql(query)
display(df)

facid,month,Total Slots
0,7,270
0,8,459
0,9,591
1,7,207
1,8,483
1,9,588
2,7,180
2,8,459
2,9,570
3,7,104


### Question 5
Find the total number of members (including guests) who have made at least one booking.
https://pgexercises.com/questions/aggregates/members1.html

In [0]:
# Define the SQL query
query = """
select count(distinct bookings.memid) as `count`
from bookings
"""

# Execute the query
df = spark.sql(query)
display(df)

count
30


###Question 6
Find the total number of members (including guests) who have made at least one booking.
https://pgexercises.com/questions/aggregates/nbooking.html

In [0]:
query = """
select surname, firstname, members.memid, min(bookings.starttime) as starttime
from members
full join bookings
on members.memid=bookings.memid
where starttime>'2012-09-01'
group by surname, firstname, members.memid
order by members.memid
"""

# Execute the query
df = spark.sql(query)
display(df)

surname,firstname,memid,starttime
GUEST,GUEST,0,2012-09-01T08:00:00.000+0000
Smith,Darren,1,2012-09-01T09:00:00.000+0000
Owen,Charles,10,2012-09-01T11:00:00.000+0000
Jones,David,11,2012-09-01T09:30:00.000+0000
Baker,Anne,12,2012-09-01T14:30:00.000+0000
Farrell,Jemima,13,2012-09-01T09:30:00.000+0000
Smith,Jack,14,2012-09-01T11:00:00.000+0000
Bader,Florence,15,2012-09-01T10:30:00.000+0000
Baker,Timothy,16,2012-09-01T15:00:00.000+0000
Pinker,David,17,2012-09-01T08:30:00.000+0000


### String & Date
### Question 1
Output the names of all members, formatted as 'Surname, Firstname'

https://pgexercises.com/questions/string/concat.html 

In [0]:

query = """
select surname || ', ' || firstname as name from members     
"""

# Execute the query
df = spark.sql(query)
display(df)

name
"GUEST, GUEST"
"Smith, Darren"
"Smith, Tracy"
"Rownam, Tim"
"Joplette, Janice"
"Butters, Gerald"
"Tracy, Burton"
"Dare, Nancy"
"Boothe, Tim"
"Stibbons, Ponder"


### Question 2
Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.
https://pgexercises.com/questions/string/case.html 

In [0]:
query = """
select * from facilities where upper(name) like 'TENNIS%'
"""

# Execute the query
df = spark.sql(query)
display(df)
  

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200


### Question 3
You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.

https://pgexercises.com/questions/string/reg.html 

In [0]:
query = """
select memid, telephone from members
where telephone like '(%%%)%%%-%%%'
"""

# Execute the query
df = spark.sql(query)
display(df)

memid,telephone
0,(000) 000-0000
3,(844) 693-0723
4,(833) 942-4710
5,(844) 078-4130
6,(822) 354-9973
7,(833) 776-4001
8,(811) 433-2547
9,(833) 160-3900
10,(855) 542-5251
11,(844) 536-8036


### Question 4
You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.

https://pgexercises.com/questions/string/substr.html 




In [0]:
query = """
select substr(surname, 1,1) as letter, count(*) as count
from members
group by letter
order by letter

"""

# Execute the query
df = spark.sql(query)
display(df)

letter,count
B,5
C,2
D,1
F,2
G,2
H,1
J,3
M,1
O,1
P,2


### Question 5
Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
https://pgexercises.com/questions/date/series.html 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sequence, to_date, explode
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit

# Initialize Spark session
spark = SparkSession.builder.appName("OctoberDates").getOrCreate()

# Define the start and end date
start_date = "2012-10-01"
end_date = "2012-10-31"

# Create a DataFrame with a single row, containing a date range as an array
date_df = spark.createDataFrame([(start_date, end_date)], ["start", "end"])

# Generate the sequence of dates and explode it into individual rows
october_dates_df = date_df.select(explode(sequence(to_date(date_df.start), to_date(date_df.end))).alias("date"))

# Show the result
october_dates_df.show(truncate=False)

+----------+
|date      |
+----------+
|2012-10-01|
|2012-10-02|
|2012-10-03|
|2012-10-04|
|2012-10-05|
|2012-10-06|
|2012-10-07|
|2012-10-08|
|2012-10-09|
|2012-10-10|
|2012-10-11|
|2012-10-12|
|2012-10-13|
|2012-10-14|
|2012-10-15|
|2012-10-16|
|2012-10-17|
|2012-10-18|
|2012-10-19|
|2012-10-20|
+----------+
only showing top 20 rows



### Question 5
Return a count of bookings for each month, sorted by month
https://pgexercises.com/questions/date/bookingspermonth.html

In [0]:
query = """
select date_trunc('month', starttime) as month, count(*)
	from bookings
	group by month
	order by month  
"""

# Execute the query
df = spark.sql(query)
display(df)

month,count(1)
2012-07-01T00:00:00.000+0000,658
2012-08-01T00:00:00.000+0000,1472
2012-09-01T00:00:00.000+0000,1913
2013-01-01T00:00:00.000+0000,1
